In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Install langchain

In [4]:
!pip install  langchain sentence_transformers -q

Install  unstructured

In [7]:
!pip install unstructured -q

Install the environment dependencies

In [6]:
!pip install pydantic==1.10.8
!pip install typing-inspect==0.8.0 typing_extensions==4.5.
!pip install chromadb==0.3.26

  Using cached typing_inspect-0.8.0-py3-none-any.whl (8.7 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: typing-inspect
    Found existing installation: typing-inspect 0.9.0
    Uninstalling typing-inspect-0.9.0:
      Successfully uninstalled typing-inspect-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.30 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
emoji 2.12.1 requires typing-extensions>=4.7.0, but you have typing-extensions 4.5.0 which is incompatible.
fastapi 0.111.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which i

Take pdf

In [8]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

Use text splitter to split text in chunks

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

89


embeddings using langchain

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using chromadb as a vectorstore and storing the docs in it

In [11]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

Doing similarity search  using query

In [12]:
query = "consequences of climate change"
matching_docs = db.similarity_search(query)

matching_docs[0]

Document(page_content='The climate and natural capital pillar looks at the impact of cooperation on the remediation and resolution of challenges to the global climate and natural capital. The focus is on lowering emissions, preserving natural capital and preparing for the likely impact of climate change through shared global goals that increase humanity’s ability to limit and adapt to a changing climate.\n\nClimate and natural capital is the sole pillar in which the majority of indicators rose across the entire period of 2012-2022, with sustained positive trends in financial commitments to mitigation and adaptation and a significant expansion of marine- protected areas. But, emissions continue to increase and progress toward ecological outcomes is stagnant (Figure 5).\n\nF I G U R E 5\n\nPillar 3: Climate and natural capital\n\nIndex [2020=1]\n\nCompound annual growth rate (CAGR) %\n\n0\n\n5\n\n10\n\n1.1\n\nClimate and natural capital index\n\n1.0\n\nMitigation ﬁnance\n\nAdaptation ﬁna

create a persist dir to store the docs

In [13]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [14]:
new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [15]:
matching_docs = new_db.similarity_search_with_score(query)
matching_docs

[(Document(page_content='The climate and natural capital pillar looks at the impact of cooperation on the remediation and resolution of challenges to the global climate and natural capital. The focus is on lowering emissions, preserving natural capital and preparing for the likely impact of climate change through shared global goals that increase humanity’s ability to limit and adapt to a changing climate.\n\nClimate and natural capital is the sole pillar in which the majority of indicators rose across the entire period of 2012-2022, with sustained positive trends in financial commitments to mitigation and adaptation and a significant expansion of marine- protected areas. But, emissions continue to increase and progress toward ecological outcomes is stagnant (Figure 5).\n\nF I G U R E 5\n\nPillar 3: Climate and natural capital\n\nIndex [2020=1]\n\nCompound annual growth rate (CAGR) %\n\n0\n\n5\n\n10\n\n1.1\n\nClimate and natural capital index\n\n1.0\n\nMitigation ﬁnance\n\nAdaptation ﬁ

In [43]:
matching_docs[0][0]

Document(page_content='The climate and natural capital pillar looks at the impact of cooperation on the remediation and resolution of challenges to the global climate and natural capital. The focus is on lowering emissions, preserving natural capital and preparing for the likely impact of climate change through shared global goals that increase humanity’s ability to limit and adapt to a changing climate.\n\nClimate and natural capital is the sole pillar in which the majority of indicators rose across the entire period of 2012-2022, with sustained positive trends in financial commitments to mitigation and adaptation and a significant expansion of marine- protected areas. But, emissions continue to increase and progress toward ecological outcomes is stagnant (Figure 5).\n\nF I G U R E 5\n\nPillar 3: Climate and natural capital\n\nIndex [2020=1]\n\nCompound annual growth rate (CAGR) %\n\n0\n\n5\n\n10\n\n1.1\n\nClimate and natural capital index\n\n1.0\n\nMitigation ﬁnance\n\nAdaptation ﬁna

In [ ]:
# prompt: (Document(page_content='The climate and natural capital pillar looks at the impact of cooperation on the remediation and resolution of challenges to the global climate and natural capital. The focus is on lowering emissions, preserving natural capital and preparing for the likely impact of climate change through shared global goals that increase humanity’s ability to limit and adapt to a changing

query = "consequences of climate change"

matching_docs = new_db.similarity_search(query)

for doc in matching_docs:
  if doc.page_content.startswith("The climate and natural capital pillar looks at the impact of cooperation on the remediation and resolution of challenges to the global climate and natural capital."):
    print(doc.page_content)
    break


MBZUAI/LaMini-T5-738M

In [16]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="MBZUAI/LaMini-T5-738M")

Use a pipeline as a high-level helper


In [28]:
db_data = vectordb.as_retriever()

In [19]:
from langchain.chains import RetrievalQA

In [20]:
from langchain_community.llms import HuggingFacePipeline

In [26]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model=base_model,
        tokenizer=tokenizer,
        max_length=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [25]:
tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-T5-738M")
base_model = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-T5-738M")

In [30]:
def qa(instruction):
  llm = llm_pipeline()
  db = Chroma(persist_directory="chroma_db", embedding_function=embeddings)
  retriever = db.as_retriever()
  qa = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=db_data , return_source_documents=True)
  return qa




In [32]:
generated_text = qa("what is climate change ")


In [33]:
generated_text

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7fd68fc53220>)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fd6863b7ee0>))